# 🏨 Hotel Data Analysis with DuckDB

ตัวอย่างการใช้ DuckDB วิเคราะห์ข้อมูลโรงแรมจาก `Hotel.csv`

In [ ]:
!pip install duckdb pandas

In [ ]:

import pandas as pd
import duckdb

# โหลด CSV (ถ้าใช้ Google Colab ให้อัปโหลดไฟล์ก่อน)
hotel_df = pd.read_csv("Hotel.csv")

# ทำความสะอาด column room_cost (ลบ comma และแปลงเป็น float)
hotel_df['room_cost'] = hotel_df['room_cost'].str.replace(",", "").astype(float)

# แปลง customer_age เป็นตัวเลข
hotel_df['customer_age'] = pd.to_numeric(hotel_df['customer_age'], errors='coerce')

# เชื่อมต่อ duckdb และ register dataframe
con = duckdb.connect()
con.register("hotel", hotel_df)

hotel_df.head()


,hotel_id,hotel_name,room_type,customer_id,customer_age,customer_gender,customer_nationality,review_score,review_text,room_cost,review_date,city
0,H0004,Avada Hotel,Deluxe Room,1001,58,Male,Thai,4,โรงเเรมดีสะอาด,4350.0,4/1/2025,Trat
1,H0001,Mandarin Osiental Bangkok,Family Room,1002,27,Female,Thai,5,เป็นโรงเเรมที่ดีที่สุดในประเทศไทย,45750.0,11/1/2025,Bangkok
2,H0005,Anantara Chiang Mai Resort & Spa,Superior Room,1003,41,Male,Chinese,5,วิวสวย,25000.0,18/1/2025,Chiang Mai
3,H0005,Anantara Chiang Mai Resort & Spa,Standard Room,1004,60,Male,Thai,4,ยายชอบล่องเรือ,20000.0,25/1/2025,Chiang Mai
4,H0002,Rimping Village,Deluxe Room,1005,33,Female,American,5,บรรยากาศดีมากๆ,5800.0,30/1/2025,Chiang Mai


## Query 1: ดูข้อมูล 5 แถวแรก

In [ ]:
con.execute("""SELECT * FROM hotel LIMIT 5""").fetchdf()

,hotel_id,hotel_name,room_type,customer_id,customer_age,customer_gender,customer_nationality,review_score,review_text,room_cost,review_date,city
0,H0004,Avada Hotel,Deluxe Room,1001,58,Male,Thai,4,โรงเเรมดีสะอาด,4350.0,4/1/2025,Trat
1,H0001,Mandarin Osiental Bangkok,Family Room,1002,27,Female,Thai,5,เป็นโรงเเรมที่ดีที่สุดในประเทศไทย,45750.0,11/1/2025,Bangkok
2,H0005,Anantara Chiang Mai Resort & Spa,Superior Room,1003,41,Male,Chinese,5,วิวสวย,25000.0,18/1/2025,Chiang Mai
3,H0005,Anantara Chiang Mai Resort & Spa,Standard Room,1004,60,Male,Thai,4,ยายชอบล่องเรือ,20000.0,25/1/2025,Chiang Mai
4,H0002,Rimping Village,Deluxe Room,1005,33,Female,American,5,บรรยากาศดีมากๆ,5800.0,30/1/2025,Chiang Mai


## Query 2: ค่าเฉลี่ยราคาโรงแรมแต่ละเมือง

In [ ]:
con.execute("""
        SELECT city, AVG(room_cost) AS avg_price
        FROM hotel
        GROUP BY city
        ORDER BY avg_price DESC
    """).fetchdf()

,city,avg_price
0,Bangkok,37725.000000
1,Chiang Mai,15733.333333
2,Chonburi,7600.000000
3,Trat,2096.000000


## Query 3: คะแนนรีวิวเฉลี่ยของแต่ละโรงแรม

In [ ]:
con.execute("""
        SELECT hotel_name, ROUND(AVG(review_score),2) AS avg_review, COUNT(*) AS total_reviews
        FROM hotel
        GROUP BY hotel_name
        ORDER BY avg_review DESC
    """).fetchdf()

,hotel_name,avg_review,total_reviews
0,Mandarin Osiental Bangkok,4.90,10
1,Rimping Village,4.45,11
2,Anantara Chiang Mai Resort & Spa,4.40,10
3,Rabbit Resort Pattaya,4.33,9
4,Avada Hotel,3.20,10


## Query 4: จำนวนลูกค้าตามสัญชาติ

In [ ]:
con.execute("""
        SELECT customer_nationality, COUNT(DISTINCT customer_id) AS num_customers
        FROM hotel
        GROUP BY customer_nationality
        ORDER BY num_customers DESC
    """).fetchdf()

,customer_nationality,num_customers
0,Thai,20
1,American,7
2,British,6
3,Indian,5
4,Chinese,5
5,French,4
6,Vietnamese,3


## Query 5: ประเภทห้องที่ราคาสูงสุดเฉลี่ย

In [ ]:
con.execute("""
        SELECT room_type, AVG(room_cost) AS avg_cost
        FROM hotel
        GROUP BY room_type
        ORDER BY avg_cost DESC
    """).fetchdf()

,room_type,avg_cost
0,Suite,23500.000000
1,Family Room,19168.421053
2,Standard Room,14068.333333
3,Superior Room,11962.857143
4,Deluxe Room,8964.285714


## Query 6: อายุเฉลี่ยของลูกค้าแต่ละโรงแรม

In [ ]:
con.execute("""
        SELECT hotel_name, ROUND(AVG(CAST(customer_age AS DOUBLE)), 1) AS avg_age, COUNT(*) AS num_customers
        FROM hotel
        GROUP BY hotel_name
        ORDER BY avg_age DESC
    """).fetchdf()

,hotel_name,avg_age,num_customers
0,Anantara Chiang Mai Resort & Spa,46.0,10
1,Mandarin Osiental Bangkok,42.7,10
2,Avada Hotel,39.7,10
3,Rabbit Resort Pattaya,33.1,9
4,Rimping Village,31.3,11


## Query 7: อายุเฉลี่ยของลูกค้าตามสัญชาติ

In [ ]:
con.execute("""
        SELECT customer_nationality, ROUND(AVG(CAST(customer_age AS DOUBLE)), 1) AS avg_age, COUNT(*) AS num_customers
        FROM hotel
        GROUP BY customer_nationality
        ORDER BY avg_age DESC
    """).fetchdf()

,customer_nationality,avg_age,num_customers
0,Thai,41.7,20
1,Chinese,41.0,5
2,British,40.3,6
3,American,40.0,7
4,Vietnamese,31.7,3
5,Indian,31.2,5
6,French,28.5,4


## Query 8: อายุเฉลี่ยของลูกค้าตามเมือง

In [ ]:
con.execute("""
        SELECT city, ROUND(AVG(CAST(customer_age AS DOUBLE)), 1) AS avg_age, COUNT(*) AS num_customers
        FROM hotel
        GROUP BY city
        ORDER BY avg_age DESC
    """).fetchdf()

,city,avg_age,num_customers
0,Bangkok,42.7,10
1,Trat,39.7,10
2,Chiang Mai,38.3,21
3,Chonburi,33.1,9


## Query 9: คะแนนรีวิว เฉลี่ยตามเมือง






In [ ]:
con.execute("""
        SELECT city, ROUND(AVG(review_score),2) AS avg_review, COUNT(*) AS total_reviews
  FROM hotel
  GROUP BY city
  ORDER BY avg_review DESC;
    """).fetchdf()

,city,avg_review,total_reviews
0,Bangkok,4.90,10
1,Chiang Mai,4.43,21
2,Chonburi,4.33,9
3,Trat,3.20,10


## Query 10: ราคาห้องเฉลี่ยที่ลูกค้าแต่ละสัญชาติจ่าย

In [ ]:
con.execute("""
       SELECT customer_nationality, ROUND(AVG(room_cost),2) AS avg_room_cost, COUNT(*) AS bookings
FROM hotel
GROUP BY customer_nationality
ORDER BY avg_room_cost DESC;
    """).fetchdf()

,customer_nationality,avg_room_cost,bookings
0,British,35083.33,6
1,Chinese,22568.00,5
2,American,15141.43,7
3,Thai,12785.00,20
4,Vietnamese,11680.00,3
5,Indian,9780.00,5
6,French,7010.00,4
